In [1]:
from librispeech_reader import *
from librispeech_to_tfrecords import *

##### Create tfrecords

In [2]:
# path to the folder with wav files
part = 'test'
wav_path = '/workspace/jgusak/Data/LibriSpeech_to_classify/{}'.format(part)

# file name with saved tfrecords
tfrecord_path = '{}/wavs.tfrecord'.format(wav_path)

In [ ]:
write_tfrecords(wav_path, tfrecord_path)

#####  Read tfrecords without defining a graph
Create generator to iterate through tfrecords 

In [4]:
record_iterator  =  tf.python_io.tf_record_iterator(path=tfrecord_path)

for string_record in record_iterator:
    example = tf.train.Example()
    example.ParseFromString(string_record)
    
    
    label = example.features.feature['label'].int64_list.value[0]
    speaker = example.features.feature['speaker'].int64_list.value[0]
    sr = example.features.feature['sr'].int64_list.value[0]

    signal_string = example.features.feature['signal_raw'].bytes_list.value[0]
    signal = np.frombuffer(signal_string, dtype = np.uint8)
    
    print(label, speaker, sr)
    print(signal.shape)
    break

33 237 16000
(160000,)


##### Create dataset

In [6]:
dataset = LibriSpeechDataset(tfrecord_path=tfrecord_path)

##### Get dataset batch

In [15]:
# get a batch in the following format: tf.Example protobuf parsed from tfrecord
batch = dataset.get_example(batch_size = 10)
print(batch.keys())

batch

dict_keys(['label', 'signal_raw', 'speaker', 'sr'])


{'label': <tf.Tensor 'ParseSingleExample_8/ParseSingleExample:0' shape=() dtype=int64>,
 'signal_raw': <tf.Tensor 'ParseSingleExample_8/ParseSingleExample:1' shape=() dtype=string>,
 'speaker': <tf.Tensor 'ParseSingleExample_8/ParseSingleExample:2' shape=() dtype=int64>,
 'sr': <tf.Tensor 'ParseSingleExample_8/ParseSingleExample:3' shape=() dtype=int64>}

In [16]:
#get a batch in the following format: {key:tensor} 
batch = dataset.get_wavenet_batch(batch_size = 10)
print(batch.keys())

batch

dict_keys(['wav', 'sr', 'speaker', 'label'])


{'wav': <tf.Tensor 'Reshape_9:0' shape=(10, 40000) dtype=float32>,
 'sr': <tf.Tensor 'strided_slice_12:0' shape=(10,) dtype=int32>,
 'speaker': <tf.Tensor 'strided_slice_13:0' shape=(10,) dtype=int32>,
 'label': <tf.Tensor 'strided_slice_14:0' shape=(10,) dtype=int32>}

##### Define graph to read tfrecords and  iterate through batches

In [17]:
dataset = LibriSpeechDataset(tfrecord_path=tfrecord_path)

LENGTH = 40000
batch = dataset.get_wavenet_batch(batch_size = 50, length = LENGTH) 

# The op for initializing the variables.
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session() as sess:
    
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord = coord)
    
    for i in range(3):
    
        batch_np = sess.run(batch)
        features, labels = batch_np['wav'], batch_np['label']

        print(features.shape, labels.shape)
        
    coord.request_stop()
    coord.join(threads)

(50, 40000) (50,)
(50, 40000) (50,)
(50, 40000) (50,)
